# 1. Import and Option

In [49]:
import pandas as pd
import requests
import json

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 15)

# 2. Get api key
[Check Open Weather Map API key](https://home.openweathermap.org/api_keys)

In [50]:
# API
key = input('openweathermap에서 복사한 api key를 입력해주세요 > ')

openweathermap에서 복사한 api key를 입력해주세요 > a


# 3. Choose cities to check weather
[도시 ID 정보 : city.list.json.gz](http://bulk.openweathermap.org/sample/)

In [51]:
# 1. 날씨 확인할 도시 직접 지정
cities = [
    "Seoul,KR",
    "Tokyo,JP",
    "New York,US",
    "Tonghae,KR",
    "Seongnam-si,KR",
    "Hanam,KR",
    "Gyeongsangnam-do,KR",
]

In [52]:
# 2. 국내 모든 도시 지정 ()
import gzip
import json

with gzip.open('city.list.json.gz', 'r') as f:
    content = f.read()
    
json_str = content.decode('utf-8')
data = json.loads(json_str)

df_city = pd.DataFrame(data)
df_kr = df_city[df_city['country']=='KR']

# Dataframe의 두 문자열 Series의 원소 합치기
# https://rfriend.tistory.com/389
df_kr_city = df_kr[['name', 'country']].apply(lambda x: ','.join(x), axis=1)

cities_list = df_kr_city.to_list()
print(f"국내 날씨 정보를 불러올 수 있는 도시 수 : {len(cities_list)}개")

국내 날씨 정보를 불러올 수 있는 도시 수 : 242개


# 4. Get Dataframe from url

In [53]:
urls = []

# 도시 id 리스트 : cities, cities_list
for city in cities:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&APPID={key}"
    urls.append(url)

df = pd.DataFrame()
for url in urls:
    res = requests.get(url)
    if res.status_code != 200:
        continue

    K2C = lambda k: k - 273.15
    data = json.loads(res.text)
    dict_temp = {
        '도시': [data['name']],
        '날씨': [data['weather'][0]['description']],
        '최저기온(섭씨)': [K2C(data['main']['temp_min'])],
        '최고기온(섭씨)': [K2C(data['main']['temp_max'])],
        '습도': [data['main']['humidity']],
        '기압': [data['main']['pressure']],
        '풍향': [data['wind']['deg']],
        '풍속': [data['wind']['speed']],
    }
    print(dict_temp)
    df_temp = pd.DataFrame(dict_temp)
    df = pd.concat([df, df_temp])

df = df.reset_index(drop=True)

# 5. Check Dataframe

In [54]:
df

""
